<a href="https://colab.research.google.com/github/aymanechilah/BMN-Boundary-Matching-Network/blob/master/Copia_de_SparkOcrImageTableRecognitionWHOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json, os
import sys

if 'google.colab' in sys.modules:
    from google.colab import files

    if 'spark_ocr.json' not in os.listdir():
      license_keys = files.upload()
      os.rename(list(license_keys.keys())[0], 'spark_ocr.json')

with open('spark_ocr.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_8542 (4).json to spark_nlp_for_healthcare_spark_ocr_8542 (4).json


In [2]:
!pip install transformers

# Installing pyspark and spark-nlp
%pip install --upgrade -q pyspark==3.2.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark OCR
#! pip uninstall spark-ocr -Y
%pip install spark-ocr==$OCR_VERSION --extra-index-url=https://pypi.johnsnowlabs.com/$SPARK_OCR_SECRET --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 18.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/5.3.1-c757bdd2f8e96ff7fce8711f33cba4c9c974e823
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 652.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

## Initialization of spark session
Need specify path to `spark-ocr-assembly.jar` or `secret`

In [1]:
import json, os

with open("spark_ocr.json", 'r') as f:
  license_keys = json.load(f)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

# Defining license key-value pairs as local variables
locals().update(license_keys)

In [2]:
from pyspark import SparkConf
from sparkocr import start

# Start spark
spark = start(secret=SPARK_OCR_SECRET, nlp_version=PUBLIC_VERSION)

Spark version: 3.2.1
Spark NLP version: 5.3.1
Spark OCR version: 5.3.1



## Printed source
### Read images and display it

In [3]:
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import display_images, display_tables, display_table
from pyspark.ml import PipelineModel
import pyspark.sql.functions as f

In [ ]:
imagePath = '/content/images_tables/'

In [ ]:
image_df= spark.read.format("binaryFile").load(imagePath)
display_images(BinaryToImage().transform(image_df), "image")

In [ ]:
binary_to_image = BinaryToImage()

table_detector = ImageTableDetector.pretrained("general_model_table_detection_v2", "en", "clinical/ocr") \
    .setInputCol("image") \
    .setOutputCol("table_regions")

#table_detector = ImageDocumentRegionDetector() \
#    .pretrained("table_detection_v3", "en", "clinical/ocr") \
#    .setInputCol("image") \
#    .setOutputCol("table_regions") \
#    .setScoreThreshold(0.8)

splitter = ImageSplitRegions() \
    .setInputCol("image") \
    .setInputRegionsCol("table_regions") \
    .setOutputCol("table_image") \
    .setDropCols("image") \
    .setImageType(ImageType.TYPE_3BYTE_BGR)

text_detector = ImageTextDetectorV2.pretrained("image_text_detector_v2", "en", "clinical/ocr") \
    .setInputCol("table_image") \
    .setOutputCol("text_regions") \
    .setWithRefiner(True)

img_to_hocr = ImageToTextV2().pretrained("ocr_base_printed_v2_opt", "en", "clinical/ocr") \
    .setInputCols(["table_image"]) \
    .setOutputFormat(OcrOutputFormat.HOCR) \
    .setRegionsColumn("text_regions") \
    .setOutputCol("hocr") \
    .setGroupImages(False) \
    .setUseGPU(False)

cell_detector = ImageDocumentRegionDetector() \
    .pretrained("region_cell_detection", "en", "clinical/ocr") \
    .setInputCol("table_image") \
    .setOutputCol("cells") \
    .setScoreThreshold(0.65)

draw_regions = ImageDrawRegions() \
    .setInputCol("table_image") \
    .setInputRegionsCol("cells") \
    .setOutputCol("image_with_regions") \
    .setRectColor(Color.red)

hocr_to_table = HocrToTextTable() \
    .setInputCol("hocr") \
    .setOutputCol("tables") \
    .setUseCellsCol("cells")

pipeline = PipelineModel(stages=[
    binary_to_image,
    table_detector,
    splitter,
    text_detector,
    cell_detector,
    draw_regions,
    img_to_hocr,
    hocr_to_table
])

result = pipeline.transform(image_df).cache()
#display(result)

### Show text regions

In [ ]:
display_images(result, "image_with_regions")

### Show images with tables

In [ ]:
display_images(result, "table_image")

### Display recognized tables

In [ ]:
display_tables(result, table_col = "tables", table_index_col = "table_index")